<a href="https://colab.research.google.com/github/elliot-brooks/nlu-coursework/blob/main/src/AV_SVM_EVAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 26.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
import torch
import re
import zipfile

# Load dev data set

In [3]:
dev_corpus = pd.read_csv("dev.csv", encoding='utf-8')
dev_labels = np.array(dev_corpus['label'])
tokeniser = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


# Load Model

In [4]:
with zipfile.ZipFile('SVM_MODEL.zip', 'r') as zip_ref:
    zip_ref.extractall('SVM_MODEL')

SVM_MODEL = tf.keras.models.load_model("SVM_MODEL/content/AV_SVM_MODEL")

#Prepare data

In [5]:
def preprocess(string):
  output = str(string).lower()
  separated_string = re.sub(r'([^\w\s])', r' \1 ', str(string))
  return output

# Prepare data for Distilled Bert
def prepare_data(data) :
  data["text_1"] = data["text_1"].apply(lambda x: preprocess(x))
  data["text_2"] = data["text_2"].apply(lambda x: preprocess(x))
  concat_pairs = []
  for index, row in data.iterrows():
      concatenated_pair = row["text_1"] + " [SEP] " + row["text_2"]
      concat_pairs.append(concatenated_pair)
  return concat_pairs

concat_data = prepare_data(dev_corpus)


SEQ_LENGTH = 128
BATCH_SIZE = 32

def create_bert_embeddings_batch(texts, tokeniser, model, batch_size, seq_length) :
  embeddings = []
  for i in range(0, len(texts), batch_size) :
    batch = texts[i:i + batch_size]
    inputs = tokeniser.batch_encode_plus(batch, padding='max_length', truncation=True, return_tensors='tf', max_length=seq_length)

    # Create embeddings
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    last_hidden_state_CLS = outputs.last_hidden_state[:, 0, :]

    embeddings.append(last_hidden_state_CLS)
  return embeddings

bert_embeddings = create_bert_embeddings_batch(concat_data, tokeniser, bert_model, BATCH_SIZE, SEQ_LENGTH)

# Test Models

In [11]:
train_inputs = np.concatenate(bert_embeddings, axis=0)
predictions = SVM_MODEL.predict(train_inputs)
binary_predictions = (predictions >= 0.5).astype(int)

tf.Tensor(
[-1.93770275e-01 -5.14306948e-02 -6.56588376e-03 -1.50499389e-01
 -4.01786156e-02 -2.30055898e-01  2.14852631e-01  3.81178558e-01
 -2.30183363e-01 -4.70842689e-01  1.18063934e-01 -1.82056613e-03
  1.24132104e-01  1.32790655e-01  7.67742563e-03  2.71812022e-01
 -2.92048976e-03  4.77398276e-01  1.09845549e-01 -1.66268796e-02
 -1.29825547e-01 -5.80034375e-01  1.47198915e-01  1.91774175e-01
  3.80342118e-02 -4.48555350e-02 -8.30437392e-02 -1.65546238e-01
  3.38495702e-01 -1.51690602e-01 -3.32439207e-02  7.36865178e-02
  2.77448446e-04 -2.70773381e-01  5.37645109e-02  2.44504977e-02
  2.57887125e-01  1.21287741e-01 -5.27847148e-02  2.89442956e-01
 -1.35685056e-01  2.25003362e-02 -2.34539192e-02 -3.92535210e-01
  4.50267009e-02 -1.71695814e-01 -2.96107769e+00 -7.09213093e-02
  9.83771756e-02 -1.32754356e-01  2.04525307e-01  3.66418399e-02
 -3.23892422e-02  1.56115934e-01  3.23677838e-01  4.31264073e-01
 -3.24815184e-01  2.14402437e-01 -2.61826843e-01 -5.68016768e-02
 -2.86725685e-

#Save predictions

In [7]:
predictions_DF = pd.DataFrame(binary_predictions, columns=['prediction'])
predictions_DF.to_csv('Group_26_A.csv', index=False)